## 멀티턴 대화 관리 방법

이 노트북은 LangChain과 LLM을 활용하여 **멀티턴(다중 턴) 대화**를 관리하는 다양한 방법을 실습합니다.

> 멀티턴 대화란?  
>  한 번의 질문-응답이 아니라, 여러 번의 대화(질문-답변-질문-답변...)가 이어지는 상황에서 **이전 대화 내용을 기억하고, 그 맥락에 맞는 답변을 생성하는 기술**입니다.

아래의 4가지 방식과 실제 끝말잇기 게임 예제를 통해  
실제 서비스에 적용할 수 있는 다양한 대화 기록 관리 방법을 배웁니다.

1. Message Passing 방식
2. ChatMessageHistory 객체 사용
3. RunnableWithMessageHistory 자동 관리
4. 대화 요약 및 히스토리 축약

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

### 1. Message passing

**가장 기본적인 멀티턴 대화 관리 방법**입니다.  

- 대화 내용을 파이썬 리스트(`history_list`)에 직접 저장하고, 매번 LLM에 전체 대화 기록을 전달합니다.
- **장점:** 구조가 단순하고, 동작 원리를 이해하기 쉽습니다.
- **단점:** 대화가 길어질수록 리스트가 커지고, 관리가 번거로워집니다.
- **활용 예시:** 간단한 챗봇, 실험용 코드, 대화 기록이 짧은 경우

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{messages}"),
    ]
)

chain = prompt | chat

In [7]:
ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "내 이름은 김일남이야. 나이는 99세야.",
            ),
            ("ai",  "그렇군요. 나이가 많으시네요!"),
            ("human", "내 나이는?"),
        ],
    }
)
print(ai_msg.content)

99세이십니다.


In [9]:
history_list = []
while(True):
    user_input = input()
    if user_input == "종료": break
    history_list.append(
        (
            "human",
            user_input,
        )
    )
    print("## CHAT_HISTORY ##")
    print(history_list, "\n")
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )
    print("AI Says : ",ai_msg.content)

    history_list.append(
        (
            "ai",
            ai_msg.content,
        )
    )

## CHAT_HISTORY ##
[('human', '한국말 할 수 있어?')] 

AI Says :  네, 한국말 할 수 있습니다. 무엇을 도와드릴까요?
## CHAT_HISTORY ##
[('human', '한국말 할 수 있어?'), ('ai', '네, 한국말 할 수 있습니다. 무엇을 도와드릴까요?'), ('human', '내가 뭐라고 했지?')] 

AI Says :  방금 "한국말 할 수 있어?"라고 물어보셨습니다.


### 2. Chat history

**LangChain의 ChatMessageHistory 객체**를 활용한 멀티턴 대화 관리입니다.

- `add_user_message`, `add_ai_message` 등 메서드로 대화 기록을 쉽게 추가/관리할 수 있습니다.
- **장점:**  
  - 메시지 타입(사용자/AI)을 명확히 구분  
  - 대화 기록 관리가 편리
- **단점:**  
  - 여전히 직접 기록을 관리해야 하므로, 대화가 길어지면 불편할 수 있습니다.
- **활용 예시:** 챗봇, 상담 서비스, 대화 기록이 중요한 서비스

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(
    "내 이름은 김일남이야. 나이는 99세야"
)

chat_history.add_ai_message("그렇군요. 나이가 많으시네요!")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={})]

In [19]:
chat_history.add_user_message(
    "내 나이는?"
)

chat_history.messages

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 나이는?', additional_kwargs={}, response_metadata={})]

In [20]:
# 답변 생성(response)
response = chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

# 대화 히스토리에 답변(response) 저장
chat_history.add_ai_message(response)
print(chat_history.messages)

첫번째 답변 후 history :  [HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}), AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 나이는?', additional_kwargs={}, response_metadata={}), AIMessage(content='99세입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7d8ed818-11e6-4caa-86ef-6f402637106e-0', usage_metadata={'input_tokens': 57, 'output_tokens': 79, 'total_tokens': 136, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 74}})]



In [21]:
chat_history = ChatMessageHistory()

while(True):
    user_input = input()
    if user_input == "종료": break
    chat_history.add_user_message(user_input)
    response = chain.invoke(
        {
            "messages": chat_history.messages,
        }
    )
    chat_history.add_ai_message(response)
    print(response.content)

네, 한국말 할 수 있습니다. 무엇을 도와드릴까요?
방금 "한국말 할 수 있어?"라고 물어보셨습니다.


### 3. Automatic history management

**RunnableWithMessageHistory**를 활용하면 세션별로 대화 기록을 자동으로 관리할 수 있습니다.

- 세션 ID만 지정하면, 각 사용자의 대화 기록을 별도로 저장/관리합니다.
- **장점:**  
  - 실제 서비스(여러 사용자 동시 접속)에서 매우 유용  
  - 코드가 간결해지고, 대화 기록 관리가 자동화됨
- **단점:**  
  - 구조를 이해하려면 약간의 학습이 필요
- **활용 예시:** 실시간 상담 챗봇, 멀티유저 게임, 고객센터 등

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# 세션별 채팅 히스토리 관리
chat_histories = {}

def get_chat_history(session_id: str):
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

chain_with_message_history = RunnableWithMessageHistory(
    chain, # 실행할 Runnable 객체
    get_chat_history, # 세션 기록을 가져오는 함수
    input_messages_key="input", # 입력 메시지의 Key
    history_messages_key="chat_history", # 대화 히스토리 메시지의 Key
)

In [40]:
chain_with_message_history.invoke(
    {"input": "내 이름은 김일남이야. 나이는 99세야."},
    {"configurable": {"session_id": "kim1"}},
)

AIMessage(content='네, 알겠습니다. 김일남 님이시고 99세이시군요.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--795ef9aa-a46a-4eb4-be91-7e9d5cdf9ff8-0', usage_metadata={'input_tokens': 41, 'output_tokens': 150, 'total_tokens': 191, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 131}})

In [41]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"}, {"configurable": {"session_id": "kim1"}}
)

AIMessage(content='네, 말씀해주셨던 대로 99세이십니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ae7b5307-3a2b-494b-b795-cf4fb9081fbe-0', usage_metadata={'input_tokens': 66, 'output_tokens': 53, 'total_tokens': 119, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 38}})

In [42]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"}, {"configurable": {"session_id": "kim2"}}
)

AIMessage(content='저는 인공지능이기 때문에 당신의 나이를 알 수 없습니다. 당신이 저에게 나이를 알려주지 않는 한, 저는 그 정보를 가지고 있지 않습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--1d3d3d2c-3591-4b70-a415-6aef903c1cb7-0', usage_metadata={'input_tokens': 29, 'output_tokens': 91, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 57}})

### 4. Modifying chat history

대화가 길어질 때, **이전 대화를 요약해서 히스토리를 압축**하는 방법입니다.

- LLM의 입력 토큰 한계를 극복할 수 있습니다.
- 요약된 메시지로 히스토리를 대체하여,  
  중요한 정보만 남기고 불필요한 대화는 줄입니다.
- **장점:**  
  - 긴 대화도 효율적으로 관리  
  - LLM의 맥락 유지 능력 향상
- **활용 예시:** 장시간 상담, 장기 프로젝트 관리, 대화 요약 기능 제공 서비스

In [57]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("내 이름은 김일남이야.")
chat_history.add_ai_message("안녕하세요, 김일남님! 무엇을 도와드릴까요?")
chat_history.add_user_message("날씨 좋은 날 들을만 한 노래 추천해주세요.")
chat_history.add_ai_message("볼빨간사춘기 – 여행을 추천해요.")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='내 이름은 김일남이야.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 김일남님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='날씨 좋은 날 들을만 한 노래 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='볼빨간사춘기 – 여행을 추천해요.', additional_kwargs={}, response_metadata={})]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history, # 단일 사용자 환경
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [59]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    print("summary_message: ",summary_message)
    
    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True

chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [60]:
chain_with_summarization.invoke(
    {"input": "내 이름은?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message:  content="김일남님이 자신의 이름을 밝힌 후, 날씨 좋은 날 들을 만한 노래를 추천해달라고 요청했고, 이에 대해 볼빨간사춘기의 '여행'이 추천되었습니다." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--aa8bfdf8-56ba-4b48-80ef-55b63130a459-0' usage_metadata={'input_tokens': 71, 'output_tokens': 508, 'total_tokens': 579, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 466}}


AIMessage(content='김일남님이십니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3b3ff337-be20-423d-b6d7-142dcc2738c7-0', usage_metadata={'input_tokens': 86, 'output_tokens': 83, 'total_tokens': 169, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}})

In [61]:
chain_with_summarization.invoke(
    {"input": "그 가수는 남자인가요 여자인가요?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message:  content='The user, Kim Il-nam, asked the AI to confirm their name, to which the AI responded, "김일남님이십니다." This interaction followed a previous exchange where Kim Il-nam had revealed their name and requested a song recommendation for a nice weather day, for which Bolbbalgan4\'s \'여행\' (Travel) was suggested.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--22df2cb1-dbf1-466a-9a36-aeae78852b46-0' usage_metadata={'input_tokens': 78, 'output_tokens': 900, 'total_tokens': 978, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 826}}


AIMessage(content='볼빨간사춘기는 여성 듀오였고, 현재는 안지영님 혼자 활동하는 여성 솔로 가수입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--56c0f4a2-00bf-4902-846b-22dd32a088d8-0', usage_metadata={'input_tokens': 126, 'output_tokens': 226, 'total_tokens': 352, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 197}})

### 끝말잇기 게임

**멀티턴 대화 관리 기술을 실제 게임에 적용한 예시**입니다.

#### 게임 규칙
1. 이미 나온 단어를 다시 말하면 패배
2. 두음법칙 허용 (ex. 리→이, 력→역, 락→낙)
3. 국어사전에 존재하는 명사만 허용
4. **아무런 설명 없이, 끝말잇기 단어만 한글로 한 단어만 출력** (프롬프트에 명확히 지시)

#### 멀티턴 대화와의 연관성
- AI가 **이전 턴에 나온 단어들을 기억**해야 하므로, 멀티턴 대화 관리 기술이 필수적입니다.
- 대화 기록을 바탕으로 **중복 단어 방지, 규칙 위반 감지**가 가능합니다.
- 실제 서비스에서는 세션별로 기록을 분리해 여러 사용자가 동시에 게임을 즐길 수 있습니다.

#### 실습 포인트
- 프롬프트 설계가 중요: "설명 없이 단어만 출력"을 명확히 지시해야 LLM이 불필요한 설명을 하지 않음
- 대화 기록 요약/관리 기법을 게임에도 적용할 수 있음

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory


# 대화 기록을 저장할 히스토리 클래스 불러오기
chat_history = ChatMessageHistory()

chat_history.messages

[]

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 끝말잇기 게임을 진행하는 AI 챗봇입니다. 아래는 게임 규칙입니다. 당신과 user 의 입력에서 아래 규칙이 꼭 지켜져야 하며, 지키지 않은 사람에게 패배를 알린 뒤, 끝말잇기 게임을 종료합니다.
                1. 주어진 대화 기록에서 이미 나왔던 단어를 다시 말했을 경우 패배합니다.
                2. 두음법칙을 허용합니다. (ex. 리 -> 이, 력 -> 역, 락 -> 낙)
                3. 국어사전에 존재하는 단어이자, 명사여야 합니다.
                4. 아무런 설명 없이, 끝말잇기 단어만 한글로 한 단어만 출력하세요.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [18]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "위 채팅 메시지는 끝말잇기 게임을 진행한 대화내용입니다. 언급한 단어들만 나열하여 저장해주세요.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [19]:
while(True):
    user_input = input("🧑 YOUR TURN : ")
    if user_input == "종료": break
    response = chain_with_summarization.invoke(
                {"input": user_input},
                {"configurable": {"session_id": "unused"}},
            )
    print("🤖 AI TURN : ", response.content)

🤖 AI TURN :  비행기
🤖 AI TURN :  술래
🤖 AI TURN :  공룡
🤖 AI TURN :  기구
🤖 AI TURN :  리본
🤖 AI TURN :  드라마
🤖 AI TURN :  감자
🤖 AI TURN :  손님
🤖 AI TURN :  금붕어
